<a href="https://colab.research.google.com/github/countifs/kaggle/blob/main/L%26T_Vehicle_Loan_Default_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1. 작업환경 설정**

## **1.1 런타임 유형 변경 : None → GPU**

## **1.2 코랩 런타임 연결 유지 및 출력 삭제 (Colab)**

```html
function ClickConnect() {
    var buttons = document.querySelectorAll("colab-dialog.yes-no-dialog paper-button#cancel"); 
    buttons.forEach(function(btn) { 
        btn.click(); 
    }); 
    console.log("1분마다 자동 재연결"); 
    document.querySelector("colab-toolbar-button#connect").click(); 
} 
setInterval(ClickConnect,1000*60);
```

## **1.3 구글 드라이브 마운트**

In [1]:
import os
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## **1.4 작업경로 변경**

In [2]:
print('> 변경 전 경로 : ' , os.getcwd())
os.chdir(input())
print('> 변경 후 경로 : ' , os.getcwd())

> 변경 전 경로 :  /content
/content/drive/MyDrive/kaggle/2. L&T Vehicle Loan Default Prediction
> 변경 후 경로 :  /content/drive/MyDrive/kaggle/2. L&T Vehicle Loan Default Prediction


## **1.5 라이브러리 불러오기**

In [4]:
# loading Common Model Algorithms

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC,LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,BaggingClassifier,AdaBoostClassifier,GradientBoostingClassifier
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score,accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.datasets import load_digits
from sklearn import tree

#Visualization
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns

# **2. 데이터 준비**

In [3]:
train_df= pd.read_csv('train.csv')
test_df= pd.read_csv('test.csv')

NameError: ignored

In [ ]:
`

In [ ]:
b